In [1]:
import pandas as pd
import os
from utils.plotter import PlotResults
from utils.tools import align_predictions

In [2]:
import torch
attr = torch.load(
    'results/FEDformer_Total/interpretation/test_feature_ablation.pt'
)

In [9]:
from run import *
argv = """
--data_path Top_20.csv --model FEDformer --result_path scratch
""".split()
parser = get_parser()
args = parser.parse_args(argv)
print(args)

setting = stringify_setting(args)

Namespace(test=False, model='FEDformer', seed=7, root_path='./dataset/processed/', data_path='Top_20.csv', result_path='scratch', freq='d', no_scale=False, seq_len=14, label_len=7, pred_len=14, top_k=5, num_kernels=6, d_model=64, n_heads=4, e_layers=2, d_layers=1, d_ff=256, moving_avg=7, factor=3, distil=True, dropout=0.1, embed='timeF', activation='gelu', output_attention=False, num_workers=0, train_epochs=10, batch_size=32, patience=3, learning_rate=0.001, des='', loss='MSE', lradj='type1', use_amp=False, no_gpu=False, gpu=0, use_multi_gpu=False, devices='0,1,2,3', p_hidden_dims=[64, 64], p_hidden_layers=2, disable_progress=False)


In [11]:
from data.data_factory import AgeData
from exp.config import Split

age_data = AgeData.build(args)
total_data = age_data.read_df()
train, val, test, updated = age_data.split_data(
    total_data, Split.primary()
)

adding time index columns TimeFromStart
added time encoded known reals ['month', 'day', 'weekday'].

Train samples 12740, validation samples 560,             test samples 560, last samples 7980
637 days of training, 14 days of validation data,             14 days of test data and 385 of data after test start.

Fitting scalers on train data


In [20]:
flag = 'test'
dataset_root = os.path.join(DataConfig.root_folder, args.data_path.split('.')[0])
path = os.path.join(dataset_root, f'{flag}.pt')
ts_dataset = None
if os.path.exists(path):
    print(f'Loading dataset from {path}')
    ts_dataset = torch.load(path)

dataset, dataloader = age_data.create_tslib_timeseries(
    data=test, train=False, 
    ts_dataset=ts_dataset # using a cached dataset help speeding up
)

Loading dataset from ./dataset/processed\Top_20\test.pt


In [21]:
time_index = age_data.time_index
predictions_index = pd.DataFrame(
    dataset.ranges, 
    columns=age_data.group_ids + [time_index]
)
predictions_index[time_index] += args.pred_len + test[time_index].min()

In [14]:
past_features = age_data.static_reals + age_data.observed_reals

In [22]:
attr = torch.load(
    f"{args.result_path}/{args.model}_{args.data_path.split('.')[0]}/interpretation/{flag}_feature_ablation.pt"
)

In [25]:
type(attr[0])

numpy.ndarray

In [35]:
pred = attr[0].mean(axis=2)[:, :, -2]

In [37]:
merged = align_predictions(
    test, predictions_index, 
    [pred], age_data, upscale=False, 
    remove_negative=False
)
merged

,Date,FIPS,TimeFromStart,Cases,Predicted_Cases
0,2021-12-12,2261,651,0.0,0.002513
1,2021-12-13,2261,652,0.0,-0.000716
2,2021-12-14,2261,653,0.0,-0.000078
3,2021-12-15,2261,654,2.0,0.004564
4,2021-12-16,2261,655,5.0,0.001311
...,...,...,...,...,...
275,2021-12-21,53033,660,612.0,0.006265
276,2021-12-22,53033,661,707.0,0.006293
277,2021-12-23,53033,662,1708.0,-0.006286
278,2021-12-24,53033,663,0.0,-0.012669
